# Data sampler

In [1]:
#| default_exp data_sampler

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import os,pickle,torch,re, numpy as np
from typing import Optional, List, Dict
from itertools import chain

from transformers import BatchEncoding, AutoTokenizer

from fastcore.utils import *

from xcai.transform import PadFeatTfm,CollapseTfm
from xcai.core import store_attr

## `Configuration file`

In [52]:
import json
from xcai.config import WIKISEEALSOTITLES

config = {config_key: WIKISEEALSOTITLES(data_dir)[config_key]}
config_file = '/Users/suchith720/Projects/data/(mapped)LF-WikiSeeAlsoTitles-320K/data_category_linker.json'

if not os.path.exists(config_file):
    with open(config_file, 'w') as file:
        json.dump(config, file, indent=4)
    

## `Setup`

In [4]:
from xcai.main import *
from xcai.core import get_pkl_file

In [8]:
data_dir = '/Users/suchith720/Projects/data/'

config_key = "data_meta"
config_dir = "/Users/suchith720/Projects/mogicX/configs"

config_file = '/Users/suchith720/Projects/data/(mapped)LF-WikiSeeAlsoTitles-320K/data_category_linker.json'

In [9]:
pkl_dir = f"{data_dir}/processed/mogicX"
pkl_file = get_pkl_file(pkl_dir, 'wikiseealsotitles_data-meta_distilbert-base-uncased', 
                        use_sxc_sampler=False, use_exact=False, use_only_test=False, use_oracle=False)

In [10]:
block = build_block(pkl_file, config_file, use_sxc=False, config_key=config_key, only_test=False, 
                    main_oversample=False, n_slbl_samples=1, do_build=True, data_dir=data_dir, 
                    keep_attention_mask=False)

/Users/suchith720/Projects/pyxclib/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [11]:
block.collator.tfms.tfms[0].sampling_features = [('lbl2data,cat2lbl2data', (4,1)), ('cat2data', 1)]

In [12]:
batch = block.train.dset.one_batch(bsz=5, seed=10)

In [13]:
batch[0].keys()

dict_keys(['data_identifier', 'data_input_text', 'data_input_ids', 'data_idx', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'cat2data_idx', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2lbl2data_idx', 'cat2lbl2data_identifier', 'cat2lbl2data_input_text', 'cat2lbl2data_input_ids'])

## `XCSamplerFeatTfm`

In [371]:
#| export
class XCSamplerFeatTfm:

    def __init__(
        self,
        pad_token:Optional[int]=0,
        oversample:Optional[bool]=False,
        sampling_features:Optional[List]=None,
        **kwargs
    ):
        store_attr('sampling_features,oversample')
        self.pad_proc = PadFeatTfm(pad_tok=pad_token, in_place=False, drop=False)
        self.col_proc = CollapseTfm()

    # sample_base_feature

    def collate_feature_idx(self, x, name, sampling_name=None):
        level = name.count('2')
        o = self.pad_proc(x, prefix=f'{name}_idx', lev=level)
    
        if f'{name}_idx' in o:
            if sampling_name is not None and f'{sampling_name}_idx' not in o:
                o[f'{sampling_name}_idx'] = o.pop(f'{name}_idx')
            o = self.rename_idx_ptr(o, name, sampling_name)
            o = {f'p{k}':v for k,v in o.items()}
            
        return o

    def get_rnd_idx_from_ptr(self, x:List, n_samples:int, oversample:Optional[bool]=True):
        if oversample: return [torch.randint(i, size=(n_samples,)) if i>0 else torch.tensor([-1]) for i in x]
        else: return [torch.randperm(i)[:n_samples] if i>0 else torch.tensor([-1]) for i in x]

    def rename_idx_ptr(self, x:Dict, prefix:str, sampling_prefix:Optional[str]=None):
        prefixes = prefix.split('2')
        for i,n in enumerate(range(len(prefixes)-1,0,-1)):
            s = '2'.join(prefixes[n:])
            p = prefix if sampling_prefix is None else sampling_prefix
            x[f'{p}_{s}2ptr'] = x.pop(f'{prefix}_idx_ptr-{i+1}')
        return x

    def get_features(self, x:Dict, prefix:str):
        pat = f'^({prefix.replace(",","|")})_.*'
        return [o for o in x if re.match(pat, o)]

    def sample_batch(self, batch:List, features:List, idxs:List, level:int):
        sbatch = []
        for b,idx in zip(batch, idxs):
            sfeatures = {}
            for feature in features:
                cfeature = self.col_proc(b[feature], level)[0]
                sfeatures[feature] = [] if idx[0] == -1 else [cfeature[i] for i in idx]
            sbatch.append(sfeatures)
        return sbatch

    def remove_unwanted_ptr(self, x:Dict):
        return {k:v for k,v in x.items() if not re.match('.*_ptr-[0-9]+$', k)}
    
    def rename_keys(self, x:Dict, prefix:str):
        keys = list(x.keys())
        for k in keys:
            nk = k.split('_', maxsplit=1)[1]
            nk = f'{prefix}_{nk}'
            if nk not in x:
                x[nk] = x[k]
                del x[k]
        return x
    
    def collate_features(self, x:List, name:str, sampling_name:Optional[str]=None):
        level = name.count('2')

        o = self.pad_proc.coll_proc(x, prefix=name, lev=level)
        if f'{name}_input_ids' in o and f'{name}_attention_mask' not in o: 
            o[f'{name}_attention_mask'] = [[1]*len(i) for i in o[f'{name}_input_ids']]
        o = self.pad_proc.proc(o)
        
        o = self.rename_idx_ptr(o, name, sampling_name)
        o = self.remove_unwanted_ptr(o)
        if sampling_name is not None: o = self.rename_keys(o, sampling_name)
        return o
                
    def sample_base_feature(self, batch:List, prefix_names:str, name:str, n_sample:int, 
                            oversample:Optional[bool]=True):
        sampled_batch, sbatch = {}, {}
        
        feat_prefix = name.split('2')
        sampling_name,ptr_name = f'{feat_prefix[0]}2{feat_prefix[-1]}',feat_prefix[-1]
        
        o = self.collate_feature_idx(batch, name=name, sampling_name=sampling_name)
        
        if len(o):
            sampling_idx = self.get_rnd_idx_from_ptr(o[f'p{sampling_name}_{ptr_name}2ptr'], n_sample, oversample=oversample)
            
            sampled_batch.update(o)
            
            feats,level = self.get_features(batch[0], prefix_names), name.count('2')-1
            sbatch = self.sample_batch(batch, feats, sampling_idx, level)
        
            o = self.collate_features(sbatch, name=name, sampling_name=sampling_name)
            sampled_batch.update(o)
        
        return sampled_batch, sbatch
        
    # sample_dep_features

    def sample_sbatch(self, batch:List, features:List, n_samples:int, oversample:Optional[bool]=True):
        sbatch = []
        for b in batch:
            
            idxs = []
            for val in b[features[0]]:
                if oversample: idx = np.random.randint(len(val), size=n_samples) if len(val) > 0 else []
                else: idx = np.random.permutation(len(val))[:n_samples]
                idxs.append(idx)
            
            sfeatures = {}
            for feature in features:
                
                svalues = []
                for val,idx in zip(b[feature],idxs):
                    svalues.append([val[i] for i in idx])
                    
                sfeatures[feature] = svalues
                
            sbatch.append(sfeatures)
        return sbatch
    
    def sample_dep_features(
        self, 
        sampled_batch:List, 
        sbatch:List, 
        names:List, 
        n_samples:List, 
        oversample:Optional[bool]=True,
    ):
        for name,n_sample in zip(names,n_samples):
            sampling_name = '2'.join(name.split('2')[:2])
            o = self.collate_feature_idx(sbatch, name=name, sampling_name=sampling_name)
    
            if len(o):
                sampled_batch.update(o)
                
                feats = self.get_features(sbatch[0], name)
                o = self.sample_sbatch(sbatch, feats, n_sample, oversample=oversample)
                o = self.collate_features(o, name=name, sampling_name=sampling_name)
                sampled_batch.update(o)
    
        return sampled_batch

    # sample features

    def sample_feature(self, batch:List, names:str, n_samples:Union[int, List], oversample:Optional[bool]=False):
        feature_names = names.split(',')
        
        if isinstance(n_samples, int): 
            n_samples = (n_samples,)*len(feature_names)

        if len(feature_names) != len(n_samples):
            raise ValueError(f'`feature_names` and `n_samples` should have same length.')
        
        base_name, dep_names = feature_names[0], feature_names[1:]
        base_n_sample, dep_n_samples = n_samples[0], n_samples[1:]

        if f'{base_name}_input_ids' in batch[0]: self.add_attention_mask = True

        for p in dep_names:
            if not p.endswith(base_name): 
                raise ValueError(f'{p} does not end with the base prefix `{base_name}`.')

        sampled_batch, sbatch = self.sample_base_feature(batch, names, base_name, base_n_sample, oversample)
        return self.sample_dep_features(sampled_batch, sbatch, dep_names, dep_n_samples, oversample)

    # left-over features
    
    def process_features(self, sampled_batch:BatchEncoding, batch:BatchEncoding, names:List):
        for name in names:
            o = self.collate_features(batch, name=name)
            sampled_batch.update(o)
        return sampled_batch


[autoreload of xcai.data_sampler failed: Traceback (most recent call last):
  File "/Users/suchith720/miniconda3/envs/mogic/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/suchith720/miniconda3/envs/mogic/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/Users/suchith720/miniconda3/envs/mogic/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
    ~~~~~~^^^^^^
  File "/Users/suchith720/miniconda3/envs/mogic/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/Users/suchith720/miniconda3/envs/mogic/lib/python3.13/site-packages/IPython/extensions/aut

In [372]:
#| export
@patch
def __call__(
    self:XCSamplerFeatTfm, 
    batch:List, 
    sampling_features:Optional[List]=None,
    oversample:Optional[bool]=None,
):  
    store_attr('sampling_features,oversample', is_none=False)

    sampled_features = set()
    out = BatchEncoding({})
    for name, n_sample in self.sampling_features:
        o = self.sample_feature(batch, name, n_sample, self.oversample)
        out.update(o)

        sampled_features.update(name.split(','))

    all_features = set([k.split('_')[0] for k in batch[0].keys()])
    remaining_features = all_features.difference(sampled_features)
    out = self.process_features(out, batch, remaining_features)
    
    return out
    

### Example

In [277]:
batch = block.train.dset.one_batch(bsz=5, seed=100)

In [278]:
sampler = XCSamplerFeatTfm(pad_token=0, sampling_features=[('lbl2data,cat2lbl2data', (4,1)), ('cat2data', 1)])

In [279]:
o = sampler(batch, oversample=False)

In [280]:
o['cat2lbl_input_ids'].shape, o['cat2lbl_attention_mask'].shape

(torch.Size([10, 9]), torch.Size([10, 9]))

In [281]:
[i for i in o.keys() if 'attention_mask' in i]

['lbl2data_attention_mask',
 'cat2lbl_attention_mask',
 'cat2data_attention_mask',
 'data_attention_mask']

In [282]:
o['lbl2data_input_ids'], o['lbl2data_attention_mask']

(tensor([[  101, 17447, 12322,  7934,  2291,   102,     0,     0,     0],
         [  101, 12483,  4163,   102,     0,     0,     0,     0,     0],
         [  101,  3816,  6741,   102,     0,     0,     0,     0,     0],
         [  101, 12483,  5584,   102,     0,     0,     0,     0,     0],
         [  101, 29083,   102,     0,     0,     0,     0,     0,     0],
         [  101, 10065,  1006,  4093,  2030,  5701,  1007,   102,     0],
         [  101, 21214,  6104,   102,     0,     0,     0,     0,     0],
         [  101, 10221,  2352,  3181,  2212,   102,     0,     0,     0],
         [  101, 27056,  3233,  4509,  8940,  2598,   102,     0,     0],
         [  101,  2862,  1997,  4865,  1998,  4731,  1999,  3163,   102],
         [  101, 12849, 17112,  5313,  3375,   102,     0,     0,     0]]),
 tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0

In [84]:
def func():
    import pdb; pdb.set_trace()
    o = sampler(batch, oversample=False)
    

In [85]:
func()

> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_39817/38607290.py(2)func()
      1 def func():
----> 2     import pdb; pdb.set_trace()
      3     o = sampler(batch, oversample=False)
      4 



ipdb>  sampler.sample_batch


<bound method XCSamplerFeatTfm.sample_batch of <__main__.XCSamplerFeatTfm object at 0x16921acf0>>


ipdb>  b sampler.sample_batch


Breakpoint 1 at /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_39817/1038120212.py:4


...

ipdb>  


> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_39817/1038120212.py(9)sample_batch()
      7         for feature in features:
      8             cfeature = self.col_proc(b[feature], level)[0]
----> 9             sfeatures[feature] = [] if idx[0] == -1 else [cfeature[i] for i in idx]
     10         sbatch.append(sfeatures)
     11     return sbatch



ipdb>  


> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_39817/1038120212.py(9)sample_batch()
      7         for feature in features:
      8             cfeature = self.col_proc(b[feature], level)[0]
----> 9             sfeatures[feature] = [] if idx[0] == -1 else [cfeature[i] for i in idx]
     10         sbatch.append(sfeatures)
     11     return sbatch



ipdb>  q


In [30]:
from tqdm.auto import tqdm

In [46]:
for batch in tqdm(block.train.dl):
    a = torch.where(batch['cat2lbl_input_ids'] == 102)[1] + 1 
    b = batch['cat2lbl_attention_mask'].sum(dim=1)
    assert torch.all(a == b).item()


  0%|          | 0/434 [00:00<?, ?it/s]

### XCSamplerFeatTfm code

In [ ]:
pad_proc = PadFeatTfm(pad_tok=0, in_place=False, drop=False)
col_proc = CollapseTfm()

In [ ]:
prefix = 'lbl2data,cat2lbl2data'
n_samples = [2,3]

names = prefix.split(',')
base_name, smp_names = names[0], names[1:]

for p in smp_names:
    if not p.endswith(base_name): 
        raise ValueError(f'{p} does not end with the base prefix `{base_name}`.')
        

In [ ]:
if isinstance(n_samples, int): 
    n_samples = (n_samples,)*len(smp_names)

if len(names) != len(n_samples):
    raise ValueError(f'`prefixes` and `n_samples` should have same length.')
    

In [ ]:
sampled_batch = {}

In [ ]:
name, n_sample = base_name, n_samples[0]

feat_prefix = name.split('2')
sampling_name,ptr_name = f'{feat_prefix[0]}2{feat_prefix[-1]}',feat_prefix[-1]


o = collate_feature_idx(batch, name=name, sampling_name=sampling_name)
sampling_idx = get_rnd_idx_from_ptr(o[f'p{sampling_name}_{ptr_name}2ptr'], n_sample, oversample=True)

sampled_batch.update(o)

In [ ]:
feats,level = get_features(batch[0], prefix), name.count('2')-1
sbatch = sample_batch(batch, feats, sampling_idx, level)

In [ ]:
o = collate_features(sbatch, name=name, sampling_name=sampling_name)
sampled_batch.update(o)

In [ ]:
name, n_sample = smp_names[0], n_samples[0]
sampling_name = '2'.join(name.split('2')[:2])

o = collate_feature_idx(batch, name=name, sampling_name=sampling_name)
sampled_batch.update(o)

feats = get_features(sbatch[0], name)

In [ ]:
o = sample_sbatch(sbatch, feats, n_samples[1], oversample=True)
o = collate_features(o, name=name, sampling_name=sampling_name)
sampled_batch.update(o)

In [ ]:
sampled_batch

{'plbl2data_data2ptr': tensor([ 1, 14,  1,  1,  1]),
 'lbl2data_idx': tensor([ 97475,  97475, 134705,  14241, 196033, 196033,  26569,  26569, 195049,
         195049]),
 'lbl2data_identifier': ['List_of_Test_cricket_umpires',
  'List_of_Test_cricket_umpires',
  'List_of_drugs_used_by_militaries',
  'Military_medicine',
  'List_of_rivers_of_Mexico',
  'List_of_rivers_of_Mexico',
  'List_of_New_South_Wales_representative_cricketers',
  'List_of_New_South_Wales_representative_cricketers',
  'List_of_antarctic_and_sub-antarctic_islands',
  'List_of_antarctic_and_sub-antarctic_islands'],
 'lbl2data_input_text': ['List of Test cricket umpires',
  'List of Test cricket umpires',
  'List of drugs used by militaries',
  'Military medicine',
  'List of rivers of Mexico',
  'List of rivers of Mexico',
  'List of New South Wales representative cricketers',
  'List of New South Wales representative cricketers',
  'List of antarctic and sub-antarctic islands',
  'List of antarctic and sub-antarctic 

## `NGAMESamplerFeatTfm`

In [285]:
#| export
class NGAMESamplerFeatTfm:

    def __init__(self, sampler_num_labels:Optional[int]=1, sampler_lbl_oversample:Optional[bool]=False, 
                 pad_token:Optional[int]=0, **kwargs):
        self.n_labels, self.lbl_oversample = sampler_num_labels, sampler_lbl_oversample
        self.pad_proc = PadFeatTfm(pad_tok=pad_token, in_place=False, drop=False)

    def sample_ids(self, x:Dict, prefix:str, idxs:torch.Tensor, data_info:Optional[Dict]=None):
        if idxs is None:
            if f'{prefix}_attention_mask' not in x: x[f'{prefix}_attention_mask'] = [[1]*len(i) for i in x[f'{prefix}_input_ids']]
        else:
            x[f'{prefix}_input_ids'] = [x[f'{prefix}_input_ids'][i] for i in idxs]
            x[f'{prefix}_attention_mask'] = [x[f'{prefix}_attention_mask'][i] for i in idxs] if f'{prefix}_attention_mask' in x else [[1]*len(i) for i in x[f'{prefix}_input_ids']]

    def align_features(self, features:List, prefix:str, level:int, idxs:Optional[torch.Tensor]=None, 
                       data_info:Optional[Dict]=None):
        if f'{prefix}_input_ids' not in features[0]: return {}

        if f'{prefix}_input_ids' in features[0] and f'{prefix}_attention_mask' in features[0]:
            features = [{f'{prefix}_input_ids': o[f'{prefix}_input_ids'], f'{prefix}_attention_mask': o[f'{prefix}_attention_mask']} for o in features]        
        else:
            features = [{f'{prefix}_input_ids': o[f'{prefix}_input_ids']} for o in features]
        
        o = self.pad_proc.coll_proc(features, prefix=prefix, lev=level)
        self.sample_ids(o, prefix, idxs, data_info)
        return self.pad_proc.proc(o)

    def sample_features(self, batch:Dict, features:List, name:str, level:int, n_samples:Optional[int]=1,
                       oversample:Optional[bool]=False, perform_cross:Optional[bool]=False):
        if f'{name}_idx' in features[0]:
            batch[f'p{name}_data2ptr'] = torch.tensor([len(o[f'{name}_idx']) for o in features], dtype=torch.int64)
            batch[f'p{name}_idx'] = torch.tensor(list(chain(*[o[f'{name}_idx'] for o in features])), dtype=torch.int64)

            indptr = torch.cat([torch.zeros((1,), dtype=torch.int64), batch[f'p{name}_data2ptr'].cumsum(dim=0)])
            if oversample: 
                idx = torch.hstack([torch.randint(n, size=(n_samples,))+offset for n,offset in zip(batch[f'p{name}_data2ptr'], indptr)])
                batch[f'{name}_data2ptr'] = torch.full(batch[f'p{name}_data2ptr'].shape, n_samples)
            else: 
                idx = torch.hstack([torch.randperm(n)[:n_samples]+offset for n,offset in zip(batch[f'p{name}_data2ptr'], indptr)])
                batch[f'{name}_data2ptr'] = torch.clamp(batch[f'p{name}_data2ptr'], max=n_samples)

            data_info = dict()
            if perform_cross:
                data_info = {
                    'data_idx': torch.arange(len(batch[f'{name}_data2ptr'])).repeat_interleave(batch[f'{name}_data2ptr']),
                    'data_input_ids': [o['data_input_ids'] for o in features],
                }
                data_info['data_attention_mask'] = [o['data_attention_mask'] for o in features] if 'data_attention_mask' in features[0] else [[1]*len(i) for i in data_info['data_input_ids']]
                
                
            batch[f'{name}_idx'] = batch[f'p{name}_idx'][idx]
            batch.update(self.align_features(features, name, level, idx, data_info))

    def collate_data(self, batch:Dict, features:List):
        if 'data_idx' in features[0]:
            batch['data_idx'] = torch.tensor([o['data_idx'] for o in features], dtype=torch.int64)
            
        o =  self.align_features(features, 'data', 0)    
        batch['data_input_ids'], batch['data_attention_mask'] = o['data_input_ids'], o['data_attention_mask']

    def collate_labels(self, batch:Dict, features:List):
        self.sample_features(batch, features, 'lbl2data', level=1, n_samples=self.n_labels, 
                             oversample=self.lbl_oversample)

    def __call__(self, features:List):
        batch = BatchEncoding({})
        self.collate_data(batch, features)
        self.collate_labels(batch, features)
        return batch


### Example

In [215]:
input = block.train.dset.one_batch(bsz=5, seed=100)

In [216]:
input[0].keys()

dict_keys(['data_identifier', 'data_input_text', 'data_input_ids', 'data_idx', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'cat2data_idx', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2lbl2data_idx', 'cat2lbl2data_identifier', 'cat2lbl2data_input_text', 'cat2lbl2data_input_ids'])

In [217]:
sampler = NGAMESamplerFeatTfm(sampler_num_labels=2, sampler_lbl_oversample=True, pad_token=0)

In [218]:
batch = sampler(input)

In [219]:
batch.keys()

dict_keys(['data_idx', 'data_input_ids', 'data_attention_mask', 'plbl2data_data2ptr', 'plbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_input_ids_ptr-1', 'lbl2data_attention_mask'])

In [220]:
from transformers import AutoTokenizer
tokz = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [221]:
batch['data_idx']

tensor([299346,  67634,  24397, 295115, 494039])

In [222]:
for o in batch['data_input_ids']: print(tokz.decode(o, skip_special_tokens=True))

dizzyfix
planetary boundary layer
plymouth colony
riverstick
standard complex


In [223]:
batch['lbl2data_data2ptr']

tensor([2, 2, 2, 2, 2])

In [224]:
for o in batch['lbl2data_input_ids']: print(tokz.decode(o, skip_special_tokens=True))

vestibular system
vestibular system
turbulence
atmospheric physics
english colonial empire
plymouth colony
list of towns and villages in ireland
list of towns and villages in ireland
koszul complex
koszul complex


## `OAKSamplerFeatTfm`

In [286]:
#| export
class OAKSamplerFeatTfm(NGAMESamplerFeatTfm):

    def __init__(
        self, 
        sampler_meta_name:Union[List, str], 
        sampler_num_meta:Optional[Union[Dict, int]]=1, 
        sampler_meta_oversample:Optional[Union[Dict, int]]=False, 
        **kwargs
    ):
        super().__init__(**kwargs)
        self.meta_name = sampler_meta_name if isinstance(sampler_meta_name, list) else [sampler_meta_name]
        self.n_meta, self.meta_oversample = sampler_num_meta, sampler_meta_oversample

    def collate_metadata(self, batch:Dict, features:List):
        for meta_name in self.meta_name:
            n_meta = self.n_meta if isinstance(self.n_meta, int) else self.n_meta.get(meta_name, 1)
            oversample = self.meta_oversample if isinstance(self.meta_oversample, int) else self.meta_oversample.get(meta_name, False)
            self.sample_features(batch, features, f'{meta_name}2data', level=1, n_samples=n_meta, oversample=oversample)

    def __call__(self, features:List):
        batch = BatchEncoding({})
        self.collate_data(batch, features)
        self.collate_labels(batch, features)
        self.collate_metadata(batch, features)
        return batch
        

### Example

In [319]:
input = block.train.dset.one_batch(bsz=5, seed=1000)

In [320]:
sampler = OAKSamplerFeatTfm(sampler_num_labels=2, sampler_meta_name='cat', sampler_num_meta=3)

In [321]:
batch = sampler(input)

In [322]:
batch.keys()

dict_keys(['data_idx', 'data_input_ids', 'data_attention_mask', 'plbl2data_data2ptr', 'plbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_input_ids_ptr-1', 'lbl2data_attention_mask', 'pcat2data_data2ptr', 'pcat2data_idx', 'cat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_input_ids_ptr-1', 'cat2data_attention_mask'])

In [323]:
from transformers import AutoTokenizer
tokz = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [324]:
batch['data_idx']

tensor([163807, 416967, 478089, 350073, 590591])

In [325]:
for o in batch['data_input_ids']: print(tokz.decode(o, skip_special_tokens=True))

konstantin muraviev
rerewhakaaitu river
ray dipierro
ordinary watercourse
t - x


In [327]:
print(batch['lbl2data_data2ptr'], end='\n\n')
for o in batch['lbl2data_input_ids']: print(tokz.decode(o, skip_special_tokens=True))

tensor([1, 1, 1, 2, 2])

bulgarian coup d ' aa©tat of 1944
list of rivers of new zealand
list of green bay packers players
main river
riparian water rights
nanotechnology in fiction
t - x


In [326]:
print(batch['cat2data_data2ptr'], end='\n\n')
for o in batch['cat2data_input_ids']: print(tokz.decode(o, skip_special_tokens=True))

tensor([3, 1, 3, 3, 3])

1893 births
1965 deaths
leaders ousted by a coup
rivers of the wellington region
1926 births
american football offensive guards
ohio state buckeyes football players
bodies of water
watercourses
flood control in the united kingdom
fictional characters with superhuman strength
terminator characters
female supervillains


## `CrossSamplerFeatTfm`

In [311]:
#| export
class CrossSamplerFeatTfm(OAKSamplerFeatTfm):

    def __init__(
        self, 
        sampler_use_sep:Optional[bool]=True, 
        **kwargs
    ):
        super().__init__(**kwargs)
        self.use_sep = sampler_use_sep

    def sample_ids(self, x:Dict, prefix:str, idxs:torch.Tensor, data_info:Optional[Dict]=None):
        if idxs is None:
            if f'{prefix}_attention_mask' not in x: x[f'{prefix}_attention_mask'] = [[1]*len(i) for i in x[f'{prefix}_input_ids']]
        else:
            if self.use_sep:
                x[f'{prefix}_input_ids'] = [data_info['data_input_ids'][i]+x[f'{prefix}_input_ids'][j][1:] for i,j in zip(data_info['data_idx'], idxs)]
                x[f'{prefix}_attention_mask'] = [data_info['data_input_ids'][i]+x[f'{prefix}_attention_mask'][j][1:] for i,j in zip(data_info['data_idx'], idxs)] if f'{prefix}_attention_mask' in x else [[1]*len(i) for i in x[f'{prefix}_input_ids']]
            else:
                x[f'{prefix}_input_ids'] = [data_info['data_input_ids'][i][:-1]+x[f'{prefix}_input_ids'][j][1:] for i,j in zip(data_info['data_idx'], idxs)]
                x[f'{prefix}_attention_mask'] = [data_info['data_input_ids'][i][:-1]+x[f'{prefix}_attention_mask'][j][1:] for i,j in zip(data_info['data_idx'], idxs)] if f'{prefix}_attention_mask' in x else [[1]*len(i) for i in x[f'{prefix}_input_ids']]

    def collate_labels(self, batch:Dict, features:List):
        self.sample_features(batch, features, 'lbl2data', level=1, n_samples=self.n_labels, 
                             oversample=self.lbl_oversample, perform_cross=True)
        
    def collate_metadata(self, batch:Dict, features:List):
        for meta_name in self.meta_name:
            n_meta = self.n_meta if isinstance(self.n_meta, int) else self.n_meta.get(meta_name, 1)
            oversample = self.meta_oversample if isinstance(self.meta_oversample, int) else self.meta_oversample.get(meta_name, False)
            self.sample_features(batch, features, f'{meta_name}2data', level=1, n_samples=n_meta, oversample=oversample, perform_cross=True)


### Examples

In [329]:
input = block.train.dset.one_batch(bsz=5, seed=1000)

In [330]:
sampler = CrossSamplerFeatTfm(sampler_num_labels=2, sampler_meta_name='cat', sampler_num_meta=3)

In [331]:
batch = sampler(input)

In [332]:
from transformers import AutoTokenizer
tokz = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [333]:
batch['data_idx']

tensor([163807, 416967, 478089, 350073, 590591])

In [334]:
for o in batch['data_input_ids']: print(tokz.decode(o, skip_special_tokens=True))

konstantin muraviev
rerewhakaaitu river
ray dipierro
ordinary watercourse
t - x


In [335]:
print(batch['lbl2data_data2ptr'], end='\n\n')
for o in batch['lbl2data_input_ids']: print(tokz.decode(o, skip_special_tokens=False))

tensor([1, 1, 1, 2, 2])

[CLS] konstantin muraviev [SEP] bulgarian coup d ' aa©tat of 1944 [SEP]
[CLS] rerewhakaaitu river [SEP] list of rivers of new zealand [SEP] [PAD]
[CLS] ray dipierro [SEP] list of green bay packers players [SEP] [PAD] [PAD] [PAD] [PAD]
[CLS] ordinary watercourse [SEP] main river [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] ordinary watercourse [SEP] riparian water rights [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] t - x [SEP] nanotechnology in fiction [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] t - x [SEP] t - x [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [336]:
print(batch['cat2data_data2ptr'], end='\n\n')
for o in batch['cat2data_input_ids']: print(tokz.decode(o, skip_special_tokens=False))

tensor([3, 1, 3, 3, 3])

[CLS] konstantin muraviev [SEP] 1893 births [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] konstantin muraviev [SEP] 1965 deaths [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] konstantin muraviev [SEP] leaders ousted by a coup [SEP] [PAD] [PAD]
[CLS] rerewhakaaitu river [SEP] rivers of the wellington region [SEP]
[CLS] ray dipierro [SEP] 1926 births [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] ray dipierro [SEP] american football offensive guards [SEP] [PAD] [PAD] [PAD] [PAD]
[CLS] ray dipierro [SEP] ohio state buckeyes football players [SEP] [PAD]
[CLS] ordinary watercourse [SEP] bodies of water [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] ordinary watercourse [SEP] watercourses [SEP] [PAD] [PAD] [PAD] [PAD]
[CLS] ordinary watercourse [SEP] flood control in the united kingdom [SEP] [PAD] [PAD]
[CLS] t - x [SEP] fictional characters with superhuman strength [SEP] [PAD] [PAD] [PAD] [PAD]
[CLS] t - x [SEP] terminator characters [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CL